In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports complete")



✓ Imports complete


## 2. Configuration


In [2]:
# Station configurations
STATION_CODES = {
    'KNYC': 'Central Park',
    'KJFK': 'JFK Airport',
    'KLGA': 'LaGuardia Airport'
}

# Keywords to identify stations in NOAA data
STATION_KEYWORDS = {
    'KNYC': ['CENTRAL PARK', 'KNYC'],
    'KJFK': ['JFK', 'JOHN F KENNEDY', 'KJFK'],
    'KLGA': ['LAGUARDIA', 'LGA', 'KLGA']
}

# Data paths
DATA_DIR = Path('../../data/noaa_asos/snow')
OUTPUT_DIR = Path('../../data/noaa_asos/snow')

print(f"Target stations: {list(STATION_CODES.values())}")
print(f"Data directory: {DATA_DIR.absolute()}")


Target stations: ['Central Park', 'JFK Airport', 'LaGuardia Airport']
Data directory: /Users/drorjac/OpenMesh_pynncml/src/analysis/snow_analysis/../../data/noaa_asos/snow


## 3. Find and Process '41...' Files Only

We only process the 417*.csv files (4177732.csv and 4177747.csv) for the selected stations.

In [3]:
# Find only the '41...' CSV files (4177732.csv and 4177747.csv)
data_files = list(DATA_DIR.glob('417*.csv'))
print(f"Found {len(data_files)} '41...' files:")
for f in sorted(data_files):
    print(f"  - {f.name}")
    
if len(data_files) == 0:
    print("⚠ Warning: No '41...' files found!")


Found 2 '41...' files:
  - 4177732.csv
  - 4177747.csv


In [4]:
# Function to read '41...' NOAA files
def read_41_file(filepath):
    """Read '41...' NOAA data file"""
    try:
        df = pd.read_csv(filepath, low_memory=False)
        print(f"\n✓ Read {filepath.name}: {len(df):,} rows, {len(df.columns)} columns")
        
        if 'STATION' in df.columns:
            stations = df['STATION'].unique()
            print(f"  Stations: {len(stations)} unique")
            if 'NAME' in df.columns:
                # Show first 3 stations
                for st in stations[:3]:
                    name = df[df['STATION'] == st]['NAME'].iloc[0] if len(df[df['STATION'] == st]) > 0 else 'Unknown'
                    print(f"    - {st}: {name}")
        else:
            print(f"  ⚠ Warning: No STATION column found")
        
        return df
    except Exception as e:
        print(f"✗ Error reading {filepath.name}: {e}")
        return None

# Read only the '41...' files
dataframes_41 = []
for filepath in data_files:
    df = read_41_file(filepath)
    if df is not None:
        dataframes_41.append(df)

print(f"\n✓ Read {len(dataframes_41)} '41...' files successfully")



✓ Read 4177732.csv: 93,166 rows, 28 columns
  Stations: 130 unique
    - US1NJUN0028: SPRINGFIELD TWP 0.7 NNE, NJ US
    - US1NYSF0158: LINDENHURST 1.0 NE, NY US
    - USW00014734: NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US

✓ Read 4177747.csv: 1,062 rows, 44 columns
  Stations: 1 unique
    - USW00014732: LAGUARDIA AIRPORT, NY US

✓ Read 2 '41...' files successfully


## 4. Process '41...' Files and Filter Selected Stations

In [5]:
# Process '41...' files and filter for selected stations
print("\n" + "="*60)
print("PROCESSING '41...' FILES")
print("="*60)

def identify_station(row):
    """Identify which of our target stations this row belongs to"""
    station_name = str(row.get('NAME', '')).upper() if pd.notna(row.get('NAME')) else ''
    station_id = str(row.get('STATION', '')).upper() if pd.notna(row.get('STATION')) else ''
    
    for code, keywords in STATION_KEYWORDS.items():
        for keyword in keywords:
            if keyword.upper() in station_name or keyword.upper() in station_id:
                return code
    return None



PROCESSING '41...' FILES


## 5. Filter and Process Selected Stations

In [6]:
# ============================================================================
# Process '41...' files and filter for selected stations
# ============================================================================

if len(dataframes_41) > 0:
    # Combine all '41...' files
    df_combined = pd.concat(dataframes_41, ignore_index=True)
    print(f"\n✓ Combined '41...' files: {len(df_combined):,} rows")
    
    # Convert DATE to datetime
    if 'DATE' in df_combined.columns:
        df_combined['DATE'] = pd.to_datetime(df_combined['DATE'], errors='coerce')
        print(f"  Converted DATE column to datetime")
    
    # Identify target stations
    df_combined['station_code'] = df_combined.apply(identify_station, axis=1)
    df_filtered = df_combined[df_combined['station_code'].notna()].copy()
    
    if len(df_filtered) > 0:
        print(f"  ✓ Filtered to target stations: {len(df_filtered):,} rows")
        
        # Show what stations were found
        station_counts = df_filtered['station_code'].value_counts()
        print(f"\n  Stations found:")
        for code, count in station_counts.items():
            station_name = STATION_CODES.get(code, code)
            print(f"    {code} ({station_name}): {count:,} rows")
        
        # Select relevant columns
        relevant_cols = ['DATE', 'station_code', 'PRCP', 'SNOW', 'SNWD', 
                        'TMAX', 'TMIN', 'TAVG', 'AWND', 'WSF2', 'WSF5', 
                        'WDF2', 'WDF5', 'STATION', 'NAME']
        
        available_cols = [col for col in relevant_cols if col in df_filtered.columns]
        df_snow = df_filtered[available_cols].copy()
        
        # Convert numeric columns
        numeric_cols = ['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TAVG', 
                       'AWND', 'WSF2', 'WSF5', 'WDF2', 'WDF5']
        for col in numeric_cols:
            if col in df_snow.columns:
                df_snow[col] = pd.to_numeric(df_snow[col], errors='coerce')
        
        print(f"\n  Selected columns: {list(df_snow.columns)}")
        print(f"✓ Processing complete")
    else:
        df_snow = None
        print("  ⚠ No target stations found in '41...' files")
else:
    df_snow = None
    print("  ⚠ No '41...' files to process")



✓ Combined '41...' files: 94,228 rows
  Converted DATE column to datetime
  ✓ Filtered to target stations: 4,158 rows

  Stations found:
    KLGA (LaGuardia Airport): 2,094 rows
    KJFK (JFK Airport): 1,032 rows
    KNYC (Central Park): 1,032 rows

  Selected columns: ['DATE', 'station_code', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TAVG', 'AWND', 'WSF2', 'WSF5', 'WDF2', 'WDF5', 'STATION', 'NAME']
✓ Processing complete


## 6. Unit Conversion and Rain Date Accumulation

### 6.1. Unit Conversion

Convert precipitation and snow from inches to mm for consistent units


In [7]:
# ============================================================================
# UNIT CONVERSION: Convert from inches to mm
# ============================================================================
print("\n" + "="*60)
print("UNIT CONVERSION: Converting from inches to mm")
print("="*60)

if df_snow is not None:
    # Convert precipitation and snow columns from inches to mm
    # Conversion factor: 1 inch = 25.4 mm
    
    # Columns to convert (precipitation and snow-related)
    columns_to_convert = ['PRCP', 'SNOW', 'SNWD']
    
    print(f"\nConverting columns from inches to mm:")
    for col in columns_to_convert:
        if col in df_snow.columns:
            # Store original stats
            non_null_count = df_snow[col].notna().sum()
            if non_null_count > 0:
                original_max = df_snow[col].max()
                original_mean = df_snow[col].mean()
                
                # Convert to mm
                df_snow[col] = df_snow[col] * 25.4
                
                # Show conversion stats
                print(f"  {col}:")
                print(f"    Original (inches): max={original_max:.2f}, mean={original_mean:.3f}")
                print(f"    Converted (mm): max={df_snow[col].max():.2f}, mean={df_snow[col].mean():.3f}")
            else:
                print(f"  {col}: No data to convert")
    
    print(f"\n✓ Unit conversion complete - all values now in mm")
else:
    print("⚠ df_snow not available for conversion")



UNIT CONVERSION: Converting from inches to mm

Converting columns from inches to mm:
  PRCP:
    Original (inches): max=8.05, mean=0.128
    Converted (mm): max=204.47, mean=3.240
  SNOW:
    Original (inches): max=6.20, mean=0.026
    Converted (mm): max=157.48, mean=0.656
  SNWD:
    Original (inches): max=5.90, mean=0.039
    Converted (mm): max=149.86, mean=0.990

✓ Unit conversion complete - all values now in mm


### 6.2. Rain Date Accumulation

Add columns to track rain dates, cumulative precipitation, and rain events


In [8]:
# ============================================================================
# RAIN DATE ACCUMULATION
# ============================================================================
print("\n" + "="*60)
print("RAIN DATE ACCUMULATION")
print("="*60)

if df_snow is not None:
    # Sort by station and date for proper accumulation
    df_snow = df_snow.sort_values(['station_code', 'DATE']).reset_index(drop=True)
    
    # Identify rain days (PRCP > 0)
    df_snow['is_rain_day'] = (df_snow['PRCP'] > 0) & (df_snow['PRCP'].notna())
    
    # Calculate cumulative precipitation per station (resets per station)
    df_snow['PRCP_cumulative'] = df_snow.groupby('station_code')['PRCP'].cumsum()
    
    # Track rain event number (each consecutive rain day gets same event number)
    df_snow['rain_event'] = 0
    for station in df_snow['station_code'].unique():
        station_mask = df_snow['station_code'] == station
        station_data = df_snow.loc[station_mask, 'is_rain_day'].copy()
        
        # Create rain events: consecutive True values are same event
        events = (station_data != station_data.shift()).cumsum()
        df_snow.loc[station_mask, 'rain_event'] = events.where(station_data, 0)
    
    # Count rain days per station
    rain_stats = df_snow.groupby('station_code').agg({
        'is_rain_day': 'sum',
        'PRCP': ['sum', 'mean', 'max'],
        'DATE': ['min', 'max']
    }).round(2)
    
    print(f"\nRain Statistics by Station:")
    print(f"{'Station':<8} {'Rain Days':<12} {'Total PRCP (mm)':<18} {'Max PRCP (mm)':<15}")
    print("-" * 60)
    for station in sorted(df_snow['station_code'].unique()):
        station_data = df_snow[df_snow['station_code'] == station]
        rain_days = station_data['is_rain_day'].sum()
        total_prcp = station_data['PRCP'].sum()
        max_prcp = station_data['PRCP'].max()
        station_name = STATION_CODES.get(station, station)
        print(f"{station:<8} {rain_days:<12} {total_prcp:<18.2f} {max_prcp:<15.2f}")
    
    # Add date range info
    print(f"\nDate Range:")
    print(f"  Min date: {df_snow['DATE'].min().date()}")
    print(f"  Max date: {df_snow['DATE'].max().date()}")
    print(f"  Total days: {(df_snow['DATE'].max() - df_snow['DATE'].min()).days + 1}")
    
    print(f"\n✓ Rain date accumulation complete")
    print(f"  Columns added: is_rain_day, PRCP_cumulative, rain_event")
else:
    print("⚠ df_snow not available for rain date accumulation")



RAIN DATE ACCUMULATION

Rain Statistics by Station:
Station  Rain Days    Total PRCP (mm)    Max PRCP (mm)  
------------------------------------------------------------
KJFK     344          2983.23            204.47         
KLGA     690          6906.01            104.14         
KNYC     343          3490.47            139.19         

Date Range:
  Min date: 2023-01-01
  Max date: 2025-11-27
  Total days: 1062

✓ Rain date accumulation complete
  Columns added: is_rain_day, PRCP_cumulative, rain_event


In [9]:
# ============================================================================
# FINAL PROCESSING - Clean and finalize df_snow
# ============================================================================
print("\n" + "="*60)
print("FINAL PROCESSING - CLEAN DATAFRAME")
print("="*60)

if df_snow is not None:
    # Ensure DATE is datetime and sorted
    df_snow['DATE'] = pd.to_datetime(df_snow['DATE'], errors='coerce')
    df_snow = df_snow.sort_values(['station_code', 'DATE']).reset_index(drop=True)
    
    # Remove duplicate rows if any
    initial_rows = len(df_snow)
    df_snow = df_snow.drop_duplicates(subset=['DATE', 'station_code'], keep='first').reset_index(drop=True)
    if len(df_snow) < initial_rows:
        print(f"  Removed {initial_rows - len(df_snow)} duplicate rows")
    
    print(f"\n✓ df_snow ready:")
    print(f"  Rows: {len(df_snow):,}")
    print(f"  Columns: {list(df_snow.columns)}")
    print(f"  Date range: {df_snow['DATE'].min().date()} to {df_snow['DATE'].max().date()}")
    print(f"  Stations: {', '.join(sorted(df_snow['station_code'].dropna().unique()))}")
    
    # Show summary
    print(f"\n📊 Data Summary:")
    for station in sorted(df_snow['station_code'].dropna().unique()):
        station_data = df_snow[df_snow['station_code'] == station]
        rain_days = station_data['is_rain_day'].sum() if 'is_rain_day' in station_data.columns else 0
        print(f"   {station} ({STATION_CODES.get(station, station)}): "
              f"{len(station_data):,} days, {rain_days} rain days")
    
    print(f"\n✓ Final dataframe ready for analysis and saving")
else:
    print("\n⚠ df_snow: Not available")



FINAL PROCESSING - CLEAN DATAFRAME
  Removed 1032 duplicate rows

✓ df_snow ready:
  Rows: 3,126
  Columns: ['DATE', 'station_code', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TAVG', 'AWND', 'WSF2', 'WSF5', 'WDF2', 'WDF5', 'STATION', 'NAME', 'is_rain_day', 'PRCP_cumulative', 'rain_event']
  Date range: 2023-01-01 to 2025-11-27
  Stations: KJFK, KLGA, KNYC

📊 Data Summary:
   KJFK (JFK Airport): 1,032 days, 344 rain days
   KLGA (LaGuardia Airport): 1,062 days, 352 rain days
   KNYC (Central Park): 1,032 days, 343 rain days

✓ Final dataframe ready for analysis and saving


## 8. Preview Data

Preview the processed dataframe


In [10]:
# Preview df_snow
if 'df_snow' in locals() and df_snow is not None:
    print("="*60)
    print("df_snow PREVIEW")
    print("="*60)
    print("\nFirst 15 rows:")
    print(df_snow.head(15))
    print(f"\nDataframe info:")
    df_snow.info()
    print(f"\nColumns: {', '.join(df_snow.columns)}")
else:
    print("⚠ df_snow not available")


df_snow PREVIEW

First 15 rows:


         DATE station_code   PRCP  SNOW  SNWD  TMAX  TMIN  TAVG   AWND  WSF2  \
0  2023-01-31         KJFK  0.762  0.00   0.0  45.0  31.0  39.0  10.29  21.0   
1  2023-02-01         KJFK  0.508  5.08   0.0  38.0  26.0  32.0  10.51  16.1   
2  2023-02-02         KJFK  0.000  0.00   0.0  39.0  24.0  32.0  10.29  23.0   
3  2023-02-03         KJFK  0.000  0.00   0.0  35.0  11.0  29.0  25.72  42.9   
4  2023-02-04         KJFK  0.000  0.00   0.0  27.0   4.0  12.0  17.00  36.9   
5  2023-02-05         KJFK  0.000  0.00   0.0  44.0  27.0  33.0  12.30  25.1   
6  2023-02-06         KJFK  0.000  0.00   0.0  53.0  32.0  42.0  12.97  31.1   
7  2023-02-07         KJFK  0.508  0.00   0.0  41.0  28.0  35.0  10.07  19.9   
8  2023-02-08         KJFK  0.254  0.00   0.0  54.0  34.0  43.0  11.41  25.9   
9  2023-02-09         KJFK  0.000  0.00   0.0  50.0  30.0  42.0   8.05  23.0   
10 2023-02-10         KJFK  0.000  0.00   0.0  60.0  41.0  52.0  15.66  29.1   
11 2023-02-11         KJFK  0.000  0.00 

In [11]:
# Summary statistics for df_snow
if 'df_snow' in locals() and df_snow is not None:
    print("\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)
    
    # Select numeric columns (exclude metadata)
    numeric_cols = df_snow.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col not in ['LATITUDE', 'LONGITUDE', 'ELEVATION']]
    
    if len(numeric_cols) > 0:
        print("\ndf_snow - Summary Statistics:")
        print(df_snow[numeric_cols].describe())
    
    # Show rain day statistics
    if 'is_rain_day' in df_snow.columns:
        print("\nRain Day Statistics:")
        print(f"  Total rain days: {df_snow['is_rain_day'].sum():,}")
        print(f"  Percentage of days with rain: {df_snow['is_rain_day'].mean()*100:.1f}%")
        
        if 'PRCP' in df_snow.columns:
            rain_only = df_snow[df_snow['is_rain_day']]
            if len(rain_only) > 0:
                print(f"  Average rain on rain days: {rain_only['PRCP'].mean():.2f} mm")
                print(f"  Max daily rainfall: {rain_only['PRCP'].max():.2f} mm")
else:
    print("⚠ df_snow not available for statistics")



SUMMARY STATISTICS

df_snow - Summary Statistics:
              PRCP         SNOW         SNWD         TMAX         TMIN  \
count  3097.000000  3096.000000  3088.000000  3096.000000  3097.000000   
mean      3.224168     0.640743     1.033931    65.284561    51.155957   
std       9.711931     6.254449     7.093941    16.608952    15.175988   
min       0.000000     0.000000     0.000000    19.000000     3.000000   
25%       0.000000     0.000000     0.000000    52.000000    39.000000   
50%       0.000000     0.000000     0.000000    66.000000    51.000000   
75%       1.270000     0.000000     0.000000    80.000000    64.000000   
max     204.470000   157.480000   149.860000   102.000000    81.000000   

              TAVG         AWND         WSF2         WSF5         WDF2  \
count  1910.000000  2811.000000  2817.000000  2815.000000  2817.000000   
mean     57.492670     8.830338    18.820767    26.798259   204.345048   
std      15.770841     4.398227     6.738706     8.695361   

In [12]:
# Save df_snow
if 'df_snow' in locals() and df_snow is not None:
    output_file = OUTPUT_DIR / 'snow_data_processed.csv'
    df_snow.to_csv(output_file, index=False)
    print("\n" + "="*60)
    print("✓ DATA SAVED")
    print("="*60)
    print(f"\n✓ Saved df_snow to: {output_file.name}")
    print(f"  Total records: {len(df_snow):,}")
    print(f"  Stations: {', '.join(sorted(df_snow['station_code'].dropna().unique()))}")
    print(f"  Columns: {', '.join(df_snow.columns)}")
    print(f"  Date range: {df_snow['DATE'].min().date()} to {df_snow['DATE'].max().date()}")
    print(f"\n✓ DATA READY FOR ANALYSIS")
    print(f"\nAvailable dataframe: df_snow")
    print(f"  Source: '41...' files (4177732.csv, 4177747.csv)")
    print(f"  Contains: PRCP, SNOW, SNWD, temperature data, and rain date accumulation")
else:
    print("\n⚠ df_snow not available to save")



✓ DATA SAVED

✓ Saved df_snow to: snow_data_processed.csv
  Total records: 3,126
  Stations: KJFK, KLGA, KNYC
  Columns: DATE, station_code, PRCP, SNOW, SNWD, TMAX, TMIN, TAVG, AWND, WSF2, WSF5, WDF2, WDF5, STATION, NAME, is_rain_day, PRCP_cumulative, rain_event
  Date range: 2023-01-01 to 2025-11-27

✓ DATA READY FOR ANALYSIS

Available dataframe: df_snow
  Source: '41...' files (4177732.csv, 4177747.csv)
  Contains: PRCP, SNOW, SNWD, temperature data, and rain date accumulation


## 9. Data Ready for Analysis

You now have two separate dataframes:

**`df_noaa_a`** - Processed files:
- Source: pcpn_noaa.csv, snow_noaa.csv, snwd_noaa.csv
- Columns: DATE, station_code, pcpn_noaa, snow_noaa, snwd_noaa
- Format: Already processed and aggregated by station

**`df_noaa_b`** - Raw NOAA files:
- Source: 417*.csv files
- Columns: DATE, station_code, PRCP, SNOW, SNWD, TMAX, TMIN, TAVG, etc.
- Format: Raw NOAA data filtered for target stations

Both dataframes can be analyzed separately or compared (e.g., pcpn_noaa vs PRCP).


## 10. Analysis

Start your analysis here using `df_noaa_a` and `df_noaa_b`


### Analysis 1: Basic Statistics by Station


In [13]:
# Basic statistics by station for df_noaa_a
if 'df_noaa_a' in locals() and df_noaa_a is not None:
    print("="*60)
    print("df_noaa_a - Statistics by Station")
    print("="*60)
    
    for station in sorted(df_noaa_a['station_code'].unique()):
        station_data = df_noaa_a[df_noaa_a['station_code'] == station]
        print(f"\n{station} ({STATION_CODES.get(station, station)}):")
        
        if 'pcpn_noaa' in station_data.columns:
            pcpn = station_data['pcpn_noaa'].dropna()
            print(f"  Precipitation (pcpn_noaa):")
            print(f"    Total: {pcpn.sum():.2f} mm")
            print(f"    Mean: {pcpn.mean():.2f} mm/day")
            print(f"    Max: {pcpn.max():.2f} mm/day")
            print(f"    Days with rain: {(pcpn > 0).sum()}")
        
        if 'snow_noaa' in station_data.columns:
            snow = station_data['snow_noaa'].dropna()
            print(f"  Snow (snow_noaa):")
            print(f"    Total: {snow.sum():.2f} mm")
            print(f"    Mean: {snow.mean():.2f} mm/day")
            print(f"    Max: {snow.max():.2f} mm/day")
            print(f"    Days with snow: {(snow > 0).sum()}")
else:
    print("df_noaa_a not available")


df_noaa_a not available


In [14]:
# Basic statistics by station for df_noaa_b
if 'df_noaa_b' in locals() and df_noaa_b is not None:
    print("="*60)
    print("df_noaa_b - Statistics by Station")
    print("="*60)
    
    for station in sorted(df_noaa_b['station_code'].dropna().unique()):
        station_data = df_noaa_b[df_noaa_b['station_code'] == station]
        print(f"\n{station} ({STATION_CODES.get(station, station)}):")
        
        if 'PRCP' in station_data.columns:
            prcp = station_data['PRCP'].dropna()
            print(f"  Precipitation (PRCP):")
            print(f"    Total: {prcp.sum():.2f} mm")
            print(f"    Mean: {prcp.mean():.2f} mm/day")
            print(f"    Max: {prcp.max():.2f} mm/day")
            print(f"    Days with rain: {(prcp > 0).sum()}")
        
        if 'SNOW' in station_data.columns:
            snow = station_data['SNOW'].dropna()
            print(f"  Snow (SNOW):")
            print(f"    Total: {snow.sum():.2f} mm")
            print(f"    Mean: {snow.mean():.2f} mm/day")
            print(f"    Max: {snow.max():.2f} mm/day")
            print(f"    Days with snow: {(snow > 0).sum()}")
else:
    print("df_noaa_b not available")


df_noaa_b not available


### Analysis 2: Compare Precipitation Data Sources


In [15]:
# Compare precipitation from df_noaa_a (pcpn_noaa) vs df_noaa_b (PRCP)
# This creates a comparison dataframe when dates overlap

if 'df_noaa_a' in locals() and 'df_noaa_b' in locals() and df_noaa_a is not None and df_noaa_b is not None:
    print("="*60)
    print("PRECIPITATION COMPARISON: pcpn_noaa (df_noaa_a) vs PRCP (df_noaa_b)")
    print("="*60)
    
    # Merge on DATE and station_code to compare
    comparison = pd.merge(
        df_noaa_a[['DATE', 'station_code', 'pcpn_noaa']],
        df_noaa_b[['DATE', 'station_code', 'PRCP']],
        on=['DATE', 'station_code'],
        how='inner'  # Only dates that exist in both
    )
    
    print(f"\nOverlapping dates: {len(comparison):,} days")
    print(f"Date range: {comparison['DATE'].min().date()} to {comparison['DATE'].max().date()}")
    
    # Calculate differences
    comparison['diff'] = comparison['pcpn_noaa'] - comparison['PRCP']
    comparison['diff_pct'] = (comparison['diff'] / comparison['PRCP'] * 100).replace([np.inf, -np.inf], np.nan)
    
    print(f"\nComparison Statistics:")
    print(f"  Mean difference: {comparison['diff'].mean():.3f} mm")
    print(f"  RMSE: {np.sqrt((comparison['diff']**2).mean()):.3f} mm")
    print(f"  Correlation: {comparison['pcpn_noaa'].corr(comparison['PRCP']):.3f}")
    
    print(f"\nBy Station:")
    for station in sorted(comparison['station_code'].unique()):
        station_comp = comparison[comparison['station_code'] == station]
        print(f"\n  {station}:")
        print(f"    Correlation: {station_comp['pcpn_noaa'].corr(station_comp['PRCP']):.3f}")
        print(f"    Mean difference: {station_comp['diff'].mean():.3f} mm")
        print(f"    RMSE: {np.sqrt((station_comp['diff']**2).mean()):.3f} mm")
    
    # Show sample comparison
    print(f"\nSample comparison (first 10 overlapping dates):")
    print(comparison[['DATE', 'station_code', 'pcpn_noaa', 'PRCP', 'diff']].head(10))
    
else:
    print("Both dataframes not available for comparison")


Both dataframes not available for comparison


### Analysis 3: Time Series Analysis


In [16]:
# Time series analysis - monthly aggregates
import matplotlib.pyplot as plt

if 'df_noaa_a' in locals() and df_noaa_a is not None:
    print("="*60)
    print("TIME SERIES ANALYSIS - df_noaa_a")
    print("="*60)
    
    # Add month and year columns
    df_noaa_a['year'] = df_noaa_a['DATE'].dt.year
    df_noaa_a['month'] = df_noaa_a['DATE'].dt.month
    
    # Monthly aggregates
    monthly_a = df_noaa_a.groupby(['year', 'month', 'station_code']).agg({
        'pcpn_noaa': 'sum',
        'snow_noaa': 'sum',
        'snwd_noaa': 'mean'
    }).reset_index()
    
    print(f"\nMonthly aggregates calculated:")
    print(f"  Total monthly records: {len(monthly_a):,}")
    print(f"\nSample monthly data:")
    print(monthly_a.head(10))
    
    # Yearly aggregates
    yearly_a = df_noaa_a.groupby(['year', 'station_code']).agg({
        'pcpn_noaa': 'sum',
        'snow_noaa': 'sum',
        'snwd_noaa': 'mean'
    }).reset_index()
    
    print(f"\nYearly aggregates:")
    print(yearly_a)
    
else:
    print("df_noaa_a not available")


df_noaa_a not available


In [17]:
# Time series analysis for df_noaa_b
if 'df_noaa_b' in locals() and df_noaa_b is not None:
    print("="*60)
    print("TIME SERIES ANALYSIS - df_noaa_b")
    print("="*60)
    
    # Add month and year columns
    df_noaa_b['year'] = df_noaa_b['DATE'].dt.year
    df_noaa_b['month'] = df_noaa_b['DATE'].dt.month
    
    # Monthly aggregates
    monthly_b = df_noaa_b.groupby(['year', 'month', 'station_code']).agg({
        'PRCP': 'sum',
        'SNOW': 'sum',
        'SNWD': 'mean',
        'TMAX': 'mean',
        'TMIN': 'mean'
    }).reset_index()
    
    print(f"\nMonthly aggregates calculated:")
    print(f"  Total monthly records: {len(monthly_b):,}")
    print(f"\nSample monthly data:")
    print(monthly_b.head(10))
    
    # Yearly aggregates
    yearly_b = df_noaa_b.groupby(['year', 'station_code']).agg({
        'PRCP': 'sum',
        'SNOW': 'sum',
        'SNWD': 'mean',
        'TMAX': 'mean',
        'TMIN': 'mean'
    }).reset_index()
    
    print(f"\nYearly aggregates:")
    print(yearly_b)
    
else:
    print("df_noaa_b not available")


df_noaa_b not available


### Analysis 4: Visualization Ready

Data is now ready for plotting. Use the monthly and yearly aggregates created above.
